In [5]:
from board_labeler import getFiles

filesDir = "labeled/"

paths = getFiles("labeled/")

In [43]:
import cv2
import re
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

stringLen = len(filesDir)
m = re.compile(r"([a-zA-Z]+(_[a-zA-Z]+)?)_")



X = [cv2.imread(x) for x in paths]
y = [m.findall(x)[0][0] for x in paths]

dictClasses = {
    "empty":0,
    "bishop_black":1,
    "bishop_white":2,
    "king_black":4,
    "king_white":5,
    "knight_black":6,
    "knight_white":7,
    "pawn_black":8,
    "pawn_white":9,
    "queen_black":10,
    "queen_white":11,
    "rook_black":12,
    "rook_white":13,
}
y = [dictClasses[x] for x in y]

temp1 = [x.shape for x in X]
temp = set(temp1)

temp3 = [temp1.count(x) for x in temp]
print(temp)
print(temp3)

print(f"{len(X)} {len(y)}")



3444 3444


In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


[8, 0, 0, 1, 6, 9, 0, 0, 0, 0, 9, 9, 0, 0, 0, 8, 0, 4, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 9, 0, 8, 0, 0, 0, 8, 10, 8, 5, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0]
